# Clustering Crypto

In [15]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [46]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [47]:
# Keep all the cryptocurrencies that are being traded.
traded_crypto_df = crypto_df[crypto_df.IsTrading == True]
traded_crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [48]:
# Keep all the cryptocurrencies that have a working algorithm.
working_crypto_df = traded_crypto_df.dropna(axis=0, subset=['Algorithm'])
working_crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [49]:
# Remove the "IsTrading" column. 
new_crypto_df = working_crypto_df.drop("IsTrading", axis=1)
new_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [50]:
# Remove rows that have at least 1 null value.
new_crypto_df.dropna(inplace=True)
new_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [51]:
# Keep the rows where coins are mined.
mined_crypto_df = new_crypto_df[new_crypto_df.TotalCoinsMined >0]
mined_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [52]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coinname_df = pd.DataFrame(mined_crypto_df["CoinName"])
coinname_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [53]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
cleaned_crypto_df = mined_crypto_df.drop(["CoinName"], axis=1)
cleaned_crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [54]:
cleaned_crypto_df.columns

Index(['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'], dtype='object')

In [55]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(data= cleaned_crypto_df, columns=["Algorithm", "ProofType"])
X.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.14159E+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11710816 -0.15287029 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.14500899 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494542  4.48942055 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217936 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694816 -0.15255997 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710535 -0.15285551 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [57]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.33789115,  1.01870612, -0.53129465],
       [-0.32110414,  1.01876493, -0.53197851],
       [ 2.31342547,  1.78261949, -0.58908688],
       ...,
       [ 0.32011926, -2.32679406,  0.38416224],
       [-0.13457595, -2.07695394,  0.40598259],
       [-0.25133049,  0.70527157, -0.30319087]])

In [58]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=X_pca,
    columns=['PC 1', 'PC 2', 'PC 3'],
    index=X.index
)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.337891,1.018706,-0.531295
404,-0.321104,1.018765,-0.531979
1337,2.313425,1.782619,-0.589087
BTC,-0.164119,-1.217666,0.176748
ETH,-0.162037,-1.977788,0.368004


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [59]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\alyss\miniconda\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1040: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [60]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)
    
# Fit the model
model.fit(pcs_df)
    
# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([3, 3, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3,
       0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0,
       0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 3, 0, 0, 0, 0, 3,
       3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 3, 0, 3, 3, 0, 3, 0, 3, 0,
       3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3,
       0, 3, 0, 0, 3, 0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 3, 0, 3, 3,
       3, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3,
       3, 0, 3, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 3, 0,
       3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 3,

In [61]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([cleaned_crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coinname_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337891,1.018706,-0.531295,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321104,1.018765,-0.531979,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11,2.313425,1.782619,-0.589087,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.164119,-1.217666,0.176748,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.162037,-1.977788,0.368004,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166035,-1.185544,-0.028905,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.401913,1.335792,-0.469448,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.166067,-2.198546,0.381350,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.160467,-1.977900,0.367968,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.134575,-2.076954,0.405983,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [64]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [63]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class]

In [65]:
# Print the total number of tradable cryptocurrencies.
print(f' There are {clustered_df["CoinName"].count()} tradable cryptocurrencies')

 There are 532 tradable cryptocurrencies


In [66]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaling_data = clustered_df.copy()
scaling_data_2 = scaling_data.drop(
    ["Algorithm", "ProofType","PC 1", "PC 2", "PC 3", "CoinName", "Class"], 
    axis=1
)
min_max = MinMaxScaler().fit_transform(scaling_data_2)
min_max

array([[0.00000000e+00, 4.20000000e-11],
       [1.06585514e-03, 5.32000000e-04],
       [2.95755050e-02, 3.14159000e-01],
       ...,
       [9.90134792e-04, 1.40022261e-03],
       [7.37027937e-06, 2.10000000e-05],
       [1.29582244e-07, 1.00000000e-06]])

In [71]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
min_max_df = pd.DataFrame(
    data=min_max,
    columns=["TotalCoinsMined", "TotalCoinSupply"],
    index=clustered_df.index
)

# Reorder columns
plot_df = min_max_df.reindex(columns=["TotalCoinSupply","TotalCoinsMined"])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = coinname_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,3
404,5.320000e-04,0.001066,404Coin,3
1337,3.141590e-01,0.029576,EliteCoin,3
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,3
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [72]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    by="Class",
    hover_cols=["CoinName"],
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)